In [2]:
from hello.hello3 import HelloApp, NotLoggedInError, BadError
import time, datetime
import requests
from hello import hello_logger
import os
import contextlib

In [19]:
err = None

def connect(ip, *args):
    global h, err
    h = HelloApp(ip, *args)
    err = None
    
def dfmt(d):
    return d.strftime("%m/%d/%Y %I:%M:%S %p")
    
def wait1(s):
    global err
    if s <= 0: return
    dend = dfmt(datetime.datetime.now() + datetime.timedelta(seconds=s))
    print("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    try:
        while time.time() < end:
            time.sleep(5)
    except KeyboardInterrupt as e:
        if err:
            raise e
        err = e
    
def wait3(s, src):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=s)
    m1 = dfmt(start)
    m2 = dfmt(end)
    print("\r(%s) Sleeping %d seconds from %s to %s          " % (src, s, m1, m2), end="")
    tend = time.time() + s
    try:
        while time.time() < tend:
            time.sleep(5)
    except KeyboardInterrupt:
        if err:
            raise e
        err = e
    
def wait2(s):
    pass
wait = wait1
    
def call(f, *a, _quiet=False, **k):
    if not _quiet:
        now = datetime.datetime.now().strftime("%H:%M:%S")
        s = "(" + ", ".join(map(repr, a)) + ")"
        print("%s Calling %s%s" % (now, f.__name__, s))
    while True:
        try:
            rv = f(*a, **k)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.Timeout as e:
            print("Got timeout! Make sure you're on the right network!")
        except (BadError, IOError, requests.exceptions.ConnectionError) as e:
            print("Error occurred:", e.__class__, e.args[0])
        else:
            return rv

def setdo(mode, n2_or_sp, o2=None):
    call(h.setdo, mode, n2_or_sp,o2)
    
def setph(mode, co2):
    call(h.setph, mode, co2, 0)
    
def phoff():
    setph(2,0)
    
def doman(n2, o2):
    setdo(1, n2, o2)
    
def doauto(sp):
    setdo(0, sp)
    
def dooff():
    setdo(2, 0, 0)
    
def phoff():
    call(h.setph, 2, 0, 0)
        
def agauto(sp):
    call(h.setag, 0, sp)
    
    
def setmg(sp):
    call(h.setmg, 1, sp)
    
def settemp(sp):
    call(h.settemp, 0, sp)
    
def setconfig(group, name, val):
    call(h.setconfig, group, name, val)

def pid1():
    setconfig("DO", "O2_P_Gain_(%/DO%)", 4)
    setconfig("DO", 'O2_I_Time_(min)', 55)
    setconfig("DO", 'O2_D_Time_(min)', 0)
    setconfig("DO", 'O2_Beta', 0)
    
    setconfig("DO", "N2_P_Gain_(%/DO%)", -20)
    setconfig("DO", 'N2_I_Time_(min)', 40)
    setconfig("DO", 'N2_D_Time_(min)', 0)
    setconfig("DO", 'N2_Beta', 0)
    
def docal1pt():
    v = call(h.getrawvalue, "doa")
    call(h.trycal, 'doa', v, 100)
    call(h.savetrialcal, 'doa')


def setup_gases():
    global do_o2_req, do_n2_req, ph_co2_req
    setconfig("pH", 'CO2_Manual_Max_(%)', 100)
    setconfig("DO", 'O2_Manual_Max_(%)', 100)
    setconfig("DO", 'N2_Manual_Max_(%)', 100)

In [39]:
def setup_logger():
    global logger
    keys = [
        'temperature.pv',
        'agitation.pv',
        'ph.outputDown',
        'ph.manDown',
        'do.manUp',
        'do.outputDown',
        'do.pv',
        'do.manDown',
        'do.outputUp',
        'maingas.man',
        'maingas.mode',
        'maingas.pv',
        'MFCs.air',
        'MFCs.n2',
        'MFCs.o2',
        'MFCs.co2'
    ]
    try:
        logger
    except NameError:
        logger = None
    if logger is not None:
        logger.stop()
    logger = hello_logger.HelloLogger(h, 10, None, keys)
    logger.start()
    
def logfile(fn):
    print("Creating new file: %r"%fn)
    logger.new_file(fn)
    
def endlog():
    logger.new_file(None)
    
def connect(ip):
    global h
    h = HelloApp(ip)
    
def endbatch():
    if call(h.batchrunning):
        call(h.endbatch)
        checkbatch(False)
        
def checkbatch(should_be=True, timeout=30):
    # the SQL actions that occur are (slightly?) asynchronous
    # when a batch is started or stopped, 
    # so subsequent rapid server calls can cause other SQL
    # requests to stall creating or retrieving the batch
    # name record. 
    # This can cause some actions to fail if there is no batch
    # registered as currently running. 
    end = time.time() + timeout
    while not (call(h.batchrunning) == should_be):
        time.sleep(1)
        if time.time() > end:
            raise TimeoutError("checkbatch failed")

def startbatch(name):
    endbatch()
    call(h.startbatch, name)
    checkbatch(True)

def mk_pid(o2p, n2p, o2i, n2i):
    def pid():
        call(h.setconfig, "DO", "O2_P_Gain_(%/DO%)", o2p)
        call(h.setconfig, "DO", 'O2_I_Time_(min)', o2i)
        call(h.setconfig, "DO", 'O2_D_Time_(min)', 0)
        call(h.setconfig, "DO", 'O2_Beta', 0)

        call(h.setconfig, "DO", "N2_P_Gain_(%/DO%)", n2p)
        call(h.setconfig, "DO", 'N2_I_Time_(min)', n2i)
        call(h.setconfig, "DO", 'N2_D_Time_(min)', 0)
        call(h.setconfig, "DO", 'N2_Beta', 0)
    return pid

def mk_test(o2p, n2p, o2i, n2i):
    n = "%d_%d_%d_%d" % (o2p, n2p, o2i, n2i)
    p = mk_pid(o2p, n2p, o2i, n2i)
    return p, n

In [37]:
def calibrate(t):
    # unlike the calibrate function used for kLa tests,
    # this one needs to flow air through the CO2 MFC
    # into the dip tube since the O2 MFC has actual oxygen
    
    setmg(0.5)
    dooff()
    setph(1, 100)
    if t < 5400:
        wait(t/2)
        phoff()
        wait(t/2)
    else:
        wait(t-3600)
        phoff()
        wait(3600)
    docal1pt()
    
def ntests(n):
    return range(1,n+1)

def _tbn(n,i,j):
    return "%s %d.%d" % (n, i, j)

@contextlib.contextmanager
def record(n1, n2, ag, mg, temp, i, j):
    id = "%d.%d"%(i,j)
    n3 = "%s_%s_%s"%(ag, mg, temp)
    logfile("3L_do_pid %s %s %s id_%s.csv" %(n1, n2, n3, id))
    startbatch("dobt %s"%id)
    try:
        yield
    finally:
        endlog()
        endbatch()

def run_test(pid, name, ag=32, mg=0.2, temp=37, ttime=3600*10):
    pid()
    agauto(ag)
    setmg(mg)
    settemp(temp)

    i=get_i()
    
    # 100% -> 150% test
    with record(name, "100_to_150", ag, mg, temp, i, 1):
        doauto(150)
        wait(ttime)

    # ramp back down 150% -> 100% (zzz...)
    with record(name, "150_to_100", ag, mg, temp, i, 2):
        doauto(100)
        wait(ttime)

    # Final test 100% -> 50%
    with record(name, "100_to_50", ag, mg, temp, i, 3):
        doauto(50)
        wait(ttime)
    
    dooff()
    
def mkfldr(p):
    path = "C:\\Users\\natha\\Documents\\test\\"+p
    os.makedirs(path, exist_ok=True)
    os.chdir(path)
    
def chd(p):
    os.chdir("C:\\Users\\natha\\Documents\\test\\"+p)
    
def get_i():
    b = call(h.getbatches)
    n = max(int(n.split()[-1].split(".")[0]) for n in filter(lambda n: n[:4] == "dobt", b.names_to_batches))
    return n + 1

In [40]:
connect("192.168.1.17")
mkfldr("do_3L_verif_ms_diptube 170920")
wait = wait1
setup_logger()
setup_gases()
calibrate(10*3600)

def test(a,b,c,d, **kw):
    p, n = mk_test(a,b,c,d)
    run_test(p, n, **kw)
    

test(4, -20, 55, 40, ag=32, temp=37, mg=0.1)
test(4, -20, 55, 40, ag=32, temp=37, mg=0.5)
dooff()
setmg(0.5)
print("Done!")

15:58:59 Calling setconfig('pH', 'CO2_Manual_Max_(%)', 100)
15:59:01 Calling setconfig('DO', 'O2_Manual_Max_(%)', 100)
15:59:01 Calling setconfig('DO', 'N2_Manual_Max_(%)', 100)
15:59:01 Calling setmg(1, 0.5)
15:59:02 Calling setdo(2, 0, 0)
15:59:03 Calling setph(1, 100, 0)
Sleeping 32400 seconds, ending at 09/21/2017 12:59:04 AM
00:59:06 Calling setph(2, 0, 0)
Error occurred: <class 'hello.hello3.ServerCallError'> Cannot perform call set - user  is missing the following permissions: Hello Access, Controls, Remote Access
Custom Error Message.vi->Determine User Permission for Call.vi->Webservice Command Handler.vi->PBS_WebServer.vi->PBS_WebServer.vi.ProxyCaller 5033
Sleeping 3600 seconds, ending at 09/21/2017 01:59:07 AM
01:59:07 Calling getRawValue('doa')
Error occurred: <class 'hello.hello3.ServerCallError'> Cannot perform call getrawvalue - user  is missing the following permissions: Hello Access, Remote Access Custom Error Message.vi->Determine User Permission for Call.vi->Webservic

# 3L Analysis

TODO - break out into different script

In [ ]:
h = HelloApp('71.189.82.196:81')
p='C:\\PBSCloudStation\\(2) R&D-Product Engineering\\Software Development\\3.0 Project\\Phase 2 Working Copy\\PID Tuning\\DO\\Raw Data\\'

def bt():
    import datetime; return datetime.datetime.now().strftime("%y%m%d")

def dl(bn):
    if isinstance(bn, str):
        b = call(h.getdatareport_bybatchname, bn)
    elif isinstance(bn, tuple):
        name, d1, d2 = bn
        b = call(h.getreport_bydate, 'process_data', d1, d2)
        bn = name
    fn = p+bn+"3L " + bt() + ".csv"
    with open(fn, 'wb') as f:
        f.write(b)
    return fn

In [ ]:
for s in (" (continued01)", 2,3):
    dl("dopid test%s"%s)

In [ ]:
from officelib.xllib import *
from officelib.const import xlconst as xlc


def open_file(file):
    global xl, wb, ws, cells, cr
    try:
        xl
    except NameError:
        xl = Excel()
    wb = xl.Workbooks.Open(file)
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cr = cells.Range

def elform(c, ref):
    a1 = c.GetAddress(0,0)
    a2 = ref.GetAddress(1,1)
    a3 = ref.Offset(0,2).GetAddress(1,1)
    form = "=(%s-%s)*%s"%(a1, a2, a3)
    return form
    
def mk_elapsed(c, ref, ins=True):
    
    if ins:
        c.Offset(1,2).EntireColumn.Insert()
    top = c.Offset(1,2)
    bot = c.End(xlc.xlDown).Offset(1,2)
    top.Value = elform(c, ref)
    r = cr(top, bot)
    top.AutoFill(r)
    r.NumberFormat = "0.00"
    
def chart_rng(c):
    x1 = c.Offset(1,2)
    x2 = x1.End(xlc.xlDown)
    y1 = c.Offset(1,3)
    y2 = y1.End(xlc.xlDown)
    return cr(x1, x2), cr(y1, y2)

def mkch(c,v):
    chart = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    mkser(chart, c,v)
    
def mkser(ch, c,v):
    x, y = chart_rng(c)
    CreateDataSeries(ch, x, y, v)
    
def fixaxes():
    for co in ws.ChartObjects():
        ax = co.Chart.Axes(xlCategory, xlPrimary)
        ax.MinimumScale = 0
    
def analyze(fn):
    c1 = cells.Find("DOPV(%)")
    ref = c1 = c1.Offset(2,1)
    mk_elapsed(c1, ref)
    ref.Offset(0, 2).Value = "24"
    mkch(c1,"DOPV(%)")
    ch2 = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    
    for v in "DON2FlowActualRequest(%)", "DOO2FlowControllerRequestLimited(%)":
        c = cells.Find(v).Offset(2,1)
        mk_elapsed(c, ref)
        mkch(c,v)
        mkser(ch2, c, v)
    fixaxes()
    wb.SaveAs(fn.replace('.csv','.xlsx'), FileFormat=xlc.xlOpenXMLWorkbook)
        
def analf(f):
    open_file(f)
    analyze(f)

In [ ]:
f = '3L_do_pid 4.-20.55.40 repeat 100 to 50.csv'
analf(os.path.abspath(f))